>Instituto Federal de Educação, Ciência e Tecnologia
>>Câmpus Campinas<br>
>>D3TOP – Tópicos em Ciência de Dados<br>
>>Prof.: Samuel Botter Martins<br>

## Preprocessing and data clean step

### Environment Config

In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
%%capture
!pip install -U spacy
!pip install -U gensim
!pip install -U neattext
!pip install -U pyspellchecker
!pip install -U stanza 
!pip install unidecode

In [4]:
# Importando bibliotecas e setando parâmetros

import numpy as np
import pandas as pd
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# seta o idioma principal do dataset a ser processado e limpo
LANG = 'english' # or 'portuguese'

#### Useful Functions

In [5]:
def remove_cols(df, mask_cmds, inverse=True):
  '''
  Remove colunas de um dado dataframe df, a partir de uma máscara booleana mask.

  Parameters:
    df (pd.Dataframe): dataframe no qual sofrera a operacao de remocao de colunas
    mask_cmds (list): array de expressões a serem avaliadas. As expressões deve retornar array de booleanos contendo True nas posições das colunas que se deseja remover
    inverse (bool): (opcional) - indica se a máscara será invertida ou não. Default: True

  Returns:
    df (pd.Dataframe): dataframe sem as colunas indicadas na máscara
  '''
  mask_exp = mask_cmds.pop(0)
  mask = eval(mask_exp)
  if inverse:
    return remove_cols(df[df.columns[~mask]], mask_cmds) if len(mask_cmds) else df[df.columns[~mask]]
  return remove_cols(df[df.columns[mask]], mask_cmds) if len(mask_cmds) else df[df.columns[mask]]

In [6]:
def missing_values_table(df):
  '''
  Funcao que retorna o calculo de valores ausentes, absoluto e percentual, das colunas de um dado dataframe pandas

  Parameters:
    df (pd.Dataframe): dataframe que será  avaliado

  Returns:
    df (pd.Dataframe): dataframe contendo nas linhas o nome das colunas do dataframe original, e nas colunas
    a quantidade total de valores ausentes e o seu percentual correspondente.
  '''
  # Total missing values
  mis_val = df.isnull().sum()
  
  # Percentage of missing values
  mis_val_percent = 100 * df.isnull().sum() / len(df)
  
  # Make a table with the results
  mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
  
  # Rename the columns
  mis_val_table_ren_columns = mis_val_table.rename(
  columns = {0 : 'Missing Values', 1 : '% of Total Values'})
  
  # Sort the table by percentage of missing descending
  mis_val_table_ren_columns = mis_val_table_ren_columns[
      mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
  '% of Total Values', ascending=False).round(1)
  
  # Print some summary information
  print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
      "There are " + str(mis_val_table_ren_columns.shape[0]) +
        " columns that have missing values.")
  
  # Return the dataframe with missing information
  return mis_val_table_ren_columns

In [7]:
def neat_preprocessing(text_in: str ) -> str:
  '''
  Funcao auxiliar que executa algumas etapas de limpeza em um dado texto usando a biblioteca neattext.
  - Remove tags HTML, URLs, pontuacoes, emails, numeros, simbolos de moeda, espaços multiplos
  - Substitue @usernames pela palavra "USER"
  - Substitue codigos de emogis pela palavra "EMOJI"

  Parameters:
    text_in (str): string de texto que sofrera o processo de limpeza.

  Returns:
    text: string de texto apos o processo de limpeza.
  '''
  import neattext.functions as ntx

  text = text_in.lower()
  
  # text = ntx.fix_contractions(text)
  text = ntx.remove_html_tags(text)
  text = ntx.remove_punctuations(text) 
  text = ntx.replace_term(text, ntx.USER_HANDLES_REGEX, 'USER')  
  text = ntx.remove_urls(text)
  text = ntx.remove_emails(text)
  text = ntx.remove_phone_numbers(text)
  text = ntx.remove_numbers(text)  
  text = ntx.replace_emojis(text, "EMOJI")
  text = ntx.remove_multiple_spaces(text)
  text = ntx.remove_currency_symbols(text)
  
  return text


In [8]:
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm

tqdm.pandas()

def parallel_applymap(df, func, worker_count, **kwargs):
  '''
  Funcao que paraleliza o metodo applymap do pandas

  Parameters:
    df (pd.DataFrame): dataframe no qual sera aplicado a funcao.
    func (callable):  funcao a ser aplicada a todas a celulas do dataframe.
    worker_count (int): quantidade de tarefas em paralelo que serão iniciadas. O dataframe sera subdividido em um numero = worker_count para cada tarefa processar.
    kwargs: demais parametros da funcao

  Returns:
    pd.Dataframe concatenado.
  '''
  def _apply(shard):
      return shard.progress_applymap(func, **kwargs)

  shards = np.array_split(df, worker_count)
  with ThreadPoolExecutor(max_workers=worker_count) as e:
      futures = e.map(_apply, shards)
  return pd.concat(list(futures))

### Data gathering

The data gathering procedure was done through [Apify](https://apify.com/dtrungtin/airbnb-scraper/input-schema#locationQuery), a web scraping and automation platform.

In [10]:
file_path = 'https://media.githubusercontent.com/media/HWatanuki/Projeto_D3TOP/main/Datasets/'
file_name = 'dataset_airbnb-scraper_2023-04-13_03-28-09-439.csv'

df_raw = pd.read_csv(file_path + file_name, dtype=str)
print(f'No Rows: {df_raw.shape[0]}')
print(f'No Columns: {df_raw.shape[1]}', end='\n\n')
display(df_raw.head(2))

No Rows: 547
No Columns: 796



,additionalHosts/0/about,additionalHosts/0/badges/0,additionalHosts/0/firstName,additionalHosts/0/hasInclusionBadge,additionalHosts/0/hasProfilePic,additionalHosts/0/hostUrl,additionalHosts/0/id,additionalHosts/0/isSuperHost,additionalHosts/0/languages/0,additionalHosts/0/languages/1,additionalHosts/0/languages/2,additionalHosts/0/languages/3,additionalHosts/0/languages/4,additionalHosts/0/memberSince,additionalHosts/0/pictureUrl,additionalHosts/0/responseRate,additionalHosts/0/responseTime,additionalHosts/0/smartName,additionalHosts/0/thumbnailUrl,additionalHosts/1/about,additionalHosts/1/firstName,additionalHosts/1/hasInclusionBadge,additionalHosts/1/hasProfilePic,additionalHosts/1/hostUrl,additionalHosts/1/id,additionalHosts/1/isSuperHost,additionalHosts/1/languages/0,additionalHosts/1/languages/1,additionalHosts/1/languages/2,additionalHosts/1/memberSince,additionalHosts/1/pictureUrl,additionalHosts/1/smartName,additionalHosts/1/thumbnailUrl,additionalHosts/2/about,additionalHosts/2/firstName,additionalHosts/2/hasInclusionBadge,additionalHosts/2/hasProfilePic,additionalHosts/2/hostUrl,additionalHosts/2/id,additionalHosts/2/isSuperHost,additionalHosts/2/languages/0,additionalHosts/2/languages/1,additionalHosts/2/memberSince,additionalHosts/2/pictureUrl,additionalHosts/2/smartName,additionalHosts/2/thumbnailUrl,address,isAvailable,isHostedBySuperhost,location/lat,location/lng,name,numberOfGuests,photos/0/caption,photos/0/pictureUrl,photos/0/thumbnailUrl,photos/1/caption,photos/1/pictureUrl,photos/1/thumbnailUrl,photos/2/caption,photos/2/pictureUrl,photos/2/thumbnailUrl,photos/3/caption,photos/3/pictureUrl,photos/3/thumbnailUrl,photos/4/caption,photos/4/pictureUrl,photos/4/thumbnailUrl,photos/5/caption,photos/5/pictureUrl,photos/5/thumbnailUrl,photos/6/caption,photos/6/pictureUrl,photos/6/thumbnailUrl,photos/7/caption,photos/7/pictureUrl,photos/7/thumbnailUrl,photos/8/caption,photos/8/pictureUrl,photos/8/thumbnailUrl,photos/9/caption,photos/9/pictureUrl,photos/9/thumbnailUrl,photos/10/caption,photos/10/pictureUrl,photos/10/thumbnailUrl,photos/11/caption,photos/11/pictureUrl,photos/11/thumbnailUrl,photos/12/caption,photos/12/pictureUrl,photos/12/thumbnailUrl,photos/13/caption,photos/13/pictureUrl,photos/13/thumbnailUrl,photos/14/caption,photos/14/pictureUrl,photos/14/thumbnailUrl,photos/15/caption,photos/15/pictureUrl,photos/15/thumbnailUrl,photos/16/caption,photos/16/pictureUrl,photos/16/thumbnailUrl,photos/17/caption,photos/17/pictureUrl,photos/17/thumbnailUrl,photos/18/caption,photos/18/pictureUrl,photos/18/thumbnailUrl,photos/19/caption,photos/19/pictureUrl,photos/19/thumbnailUrl,photos/20/caption,photos/20/pictureUrl,photos/20/thumbnailUrl,photos/21/caption,photos/21/pictureUrl,photos/21/thumbnailUrl,photos/22/caption,photos/22/pictureUrl,photos/22/thumbnailUrl,photos/23/caption,photos/23/pictureUrl,photos/23/thumbnailUrl,photos/24/caption,photos/24/pictureUrl,photos/24/thumbnailUrl,photos/25/caption,photos/25/pictureUrl,photos/25/thumbnailUrl,photos/26/caption,photos/26/pictureUrl,photos/26/thumbnailUrl,photos/27/caption,photos/27/pictureUrl,photos/27/thumbnailUrl,photos/28/caption,photos/28/pictureUrl,photos/28/thumbnailUrl,photos/29/caption,photos/29/pictureUrl,photos/29/thumbnailUrl,photos/30/caption,photos/30/pictureUrl,photos/30/thumbnailUrl,photos/31/caption,photos/31/pictureUrl,photos/31/thumbnailUrl,photos/32/caption,photos/32/pictureUrl,photos/32/thumbnailUrl,photos/33/caption,photos/33/pictureUrl,photos/33/thumbnailUrl,photos/34/caption,photos/34/pictureUrl,photos/34/thumbnailUrl,photos/35/caption,photos/35/pictureUrl,photos/35/thumbnailUrl,photos/36/caption,photos/36/pictureUrl,photos/36/thumbnailUrl,photos/37/caption,photos/37/pictureUrl,photos/37/thumbnailUrl,photos/38/caption,photos/38/pictureUrl,photos/38/thumbnailUrl,photos/39/caption,photos/39/pictureUrl,photos/39/thumbnailUrl,photos/40/caption,photos/40/pictureUrl,photos/40/thumbnailUrl,photos/41/caption,photos/41/pictureUrl,photos/41/thumbnailUrl,photo

In [ ]:
df_raw.columns[df_raw.columns.str.contains('id')]

Index(['additionalHosts/0/id', 'additionalHosts/1/id', 'additionalHosts/2/id',
       'primaryHost/id', 'reviews/0/author/id', 'reviews/0/id',
       'reviews/0/recipient/id', 'reviews/1/author/id', 'reviews/1/id',
       'reviews/1/recipient/id', 'reviews/2/author/id', 'reviews/2/id',
       'reviews/2/recipient/id', 'reviews/3/author/id', 'reviews/3/id',
       'reviews/3/recipient/id', 'reviews/4/author/id', 'reviews/4/id',
       'reviews/4/recipient/id', 'reviews/5/author/id', 'reviews/5/id',
       'reviews/5/recipient/id', 'reviews/6/author/id', 'reviews/6/id',
       'reviews/6/recipient/id', 'reviews/7/author/id', 'reviews/7/id',
       'reviews/7/recipient/id', 'reviews/8/author/id', 'reviews/8/id',
       'reviews/8/recipient/id', 'reviews/9/author/id', 'reviews/9/id',
       'reviews/9/recipient/id'],
      dtype='object')

### Step 01 - discarding information that doesn't matter

Removing columns relating to:
- `additionalHosts`
- property `photos` (we'll keep just first) 
- URL's
- `primaryHost/languages` (we'll keep just first) 

In [11]:
mask_additionalHosts = "df.columns.str.startswith('additionalHosts')"
mask_photos = "df.columns.str.contains(r'^photos/[^0]')"
mask_url = "df.columns.str.contains(r'^(?!photos).*Url$')"
mask_phlang = "df.columns.str.contains(r'^primaryHost/languages/[^0]')"
masks_commands = [mask_additionalHosts, mask_photos, mask_url, mask_phlang]

df_s01 = df_raw.pipe(remove_cols, masks_commands)

print(f'No Columns before step 01: {df_raw.shape[1]}')
print(f'No Columns after step 01: {df_s01.shape[1]}', end='\n\n')
df_s01.sample(1)

No Columns before step 01: 796
No Columns after step 01: 234



,address,isAvailable,isHostedBySuperhost,location/lat,location/lng,name,numberOfGuests,photos/0/caption,photos/0/pictureUrl,photos/0/thumbnailUrl,pricing/rate/amount,pricing/rate/amountFormatted,pricing/rate/currency,pricing/rate/isMicrosAccuracy,pricing/rateType,primaryHost/about,primaryHost/badges/0,primaryHost/badges/1,primaryHost/badges/2,primaryHost/firstName,primaryHost/hasInclusionBadge,primaryHost/hasProfilePic,primaryHost/id,primaryHost/isSuperHost,primaryHost/languages/0,primaryHost/listingsCount,primaryHost/memberSince,primaryHost/responseRate,primaryHost/responseTime,primaryHost/smartName,primaryHost/totalListingsCount,reviews/0/author/firstName,reviews/0/author/hasProfilePic,reviews/0/author/id,reviews/0/author/smartName,reviews/0/collectionTag,reviews/0/comments,reviews/0/createdAt,reviews/0/id,reviews/0/language,reviews/0/localizedDate,reviews/0/localizedReview,reviews/0/localizedReview/comments,reviews/0/localizedReview/disclaimer,reviews/0/localizedReview/needsTranslation,reviews/0/localizedReview/response,reviews/0/rating,reviews/0/recipient/firstName,reviews/0/recipient/hasProfilePic,reviews/0/recipient/id,reviews/0/recipient/smartName,reviews/0/response,reviews/1/author/firstName,reviews/1/author/hasProfilePic,reviews/1/author/id,reviews/1/author/smartName,reviews/1/collectionTag,reviews/1/comments,reviews/1/createdAt,reviews/1/id,reviews/1/localizedDate,reviews/1/localizedReview,reviews/1/rating,reviews/1/recipient/firstName,reviews/1/recipient/hasProfilePic,reviews/1/recipient/id,reviews/1/recipient/smartName,reviews/1/response,reviews/2/author/firstName,reviews/2/author/hasProfilePic,reviews/2/author/id,reviews/2/author/smartName,reviews/2/collectionTag,reviews/2/comments,reviews/2/createdAt,reviews/2/id,reviews/2/language,reviews/2/localizedDate,reviews/2/localizedReview,reviews/2/localizedReview/comments,reviews/2/localizedReview/disclaimer,reviews/2/localizedReview/needsTranslation,reviews/2/localizedReview/response,reviews/2/rating,reviews/2/recipient/firstName,reviews/2/recipient/hasProfilePic,reviews/2/recipient/id,reviews/2/recipient/smartName,reviews/2/response,reviews/3/author/firstName,reviews/3/author/hasProfilePic,reviews/3/author/id,reviews/3/author/smartName,reviews/3/collectionTag,reviews/3/comments,reviews/3/createdAt,reviews/3/id,reviews/3/language,reviews/3/localizedDate,reviews/3/localizedReview,reviews/3/localizedReview/comments,reviews/3/localizedReview/disclaimer,reviews/3/localizedReview/needsTranslation,reviews/3/localizedReview/response,reviews/3/rating,reviews/3/recipient/firstName,reviews/3/recipient/hasProfilePic,reviews/3/recipient/id,reviews/3/recipient/smartName,reviews/3/response,reviews/4/author/firstName,reviews/4/author/hasProfilePic,reviews/4/author/id,reviews/4/author/smartName,reviews/4/collectionTag,reviews/4/comments,reviews/4/createdAt,reviews/4/id,reviews/4/language,reviews/4/localizedDate,reviews/4/localizedReview,reviews/4/localizedReview/comments,reviews/4/localizedReview/disclaimer,reviews/4/localizedReview/needsTranslation,reviews/4/localizedReview/response,reviews/4/rating,reviews/4/recipient/firstName,reviews/4/recipient/hasProfilePic,reviews/4/recipient/id,reviews/4/recipient/smartName,reviews/4/response,reviews/5/author/firstName,reviews/5/author/hasProfilePic,reviews/5/author/id,reviews/5/author/smartName,reviews/5/collectionTag,reviews/5/comments,reviews/5/createdAt,reviews/5/id,reviews/5/language,reviews/5/localizedDate,reviews/5/localizedReview,reviews/5/localizedReview/comments,reviews/5/localizedReview/disclaimer,reviews/5/localizedReview/needsTranslation,reviews/5/localizedReview/response,reviews/5/rating,reviews/5/recipient/firstName,reviews/5/recipient/hasProfilePic,reviews/5/recipient/id,reviews/5/recipient/smartName,reviews/5/response,reviews/6/author/firstName,reviews/6/author/hasProfilePic,reviews/6/author/id,reviews/6/author/smartName,reviews/6/collectionTag,reviews/6/comments,reviews/6/createdAt,reviews/6/id,reviews/6/localizedDate,reviews

### Step 02 - Evaluate Missing Values

In [12]:
missing_values_table(df_s01)

Your selected dataframe has 234 columns.
There are 215 columns that have missing values.


,Missing Values,% of Total Values
reviews/4/localizedReview/response,547,100.000
reviews/7/localizedReview/response,547,100.000
reviews/1/collectionTag,547,100.000
reviews/8/localizedReview/response,547,100.000
reviews/1/localizedReview,547,100.000
reviews/8/localizedReview,547,100.000
reviews/8/collectionTag,547,100.000
reviews/2/collectionTag,547,100.000
reviews/2/localizedReview,547,100.000
reviews/2/localizedReview/response,547,100.000


### Step 03 - discarding information based on missing values

keeping only the following columns about reviews:
- Review: `comments`, `createdAt`, `id`, `rating`
- Review Author: `firstName`, `id`

In [13]:
mask_reviews_data = "df.columns.str.contains(r'^reviews/[0-9]+/(?!author/)(?!comments|createdAt|id|rating)')"
mask_author_data = "df.columns.str.contains(r'^reviews/[0-9]+/author/(?!firstName|id)')"
masks_commands = [mask_reviews_data, mask_author_data]

df_s03 = df_s01.pipe(remove_cols, masks_commands)

print(f'No Columns before step 02: {df_s01.shape[1]}')
print(f'No Columns after step 02: {df_s03.shape[1]}', end='\n\n')
df_s03.sample(1)

No Columns before step 02: 234
No Columns after step 02: 94



,address,isAvailable,isHostedBySuperhost,location/lat,location/lng,name,numberOfGuests,photos/0/caption,photos/0/pictureUrl,photos/0/thumbnailUrl,pricing/rate/amount,pricing/rate/amountFormatted,pricing/rate/currency,pricing/rate/isMicrosAccuracy,pricing/rateType,primaryHost/about,primaryHost/badges/0,primaryHost/badges/1,primaryHost/badges/2,primaryHost/firstName,primaryHost/hasInclusionBadge,primaryHost/hasProfilePic,primaryHost/id,primaryHost/isSuperHost,primaryHost/languages/0,primaryHost/listingsCount,primaryHost/memberSince,primaryHost/responseRate,primaryHost/responseTime,primaryHost/smartName,primaryHost/totalListingsCount,reviews/0/author/firstName,reviews/0/author/id,reviews/0/comments,reviews/0/createdAt,reviews/0/id,reviews/0/rating,reviews/1/author/firstName,reviews/1/author/id,reviews/1/comments,reviews/1/createdAt,reviews/1/id,reviews/1/rating,reviews/2/author/firstName,reviews/2/author/id,reviews/2/comments,reviews/2/createdAt,reviews/2/id,reviews/2/rating,reviews/3/author/firstName,reviews/3/author/id,reviews/3/comments,reviews/3/createdAt,reviews/3/id,reviews/3/rating,reviews/4/author/firstName,reviews/4/author/id,reviews/4/comments,reviews/4/createdAt,reviews/4/id,reviews/4/rating,reviews/5/author/firstName,reviews/5/author/id,reviews/5/comments,reviews/5/createdAt,reviews/5/id,reviews/5/rating,reviews/6/author/firstName,reviews/6/author/id,reviews/6/comments,reviews/6/createdAt,reviews/6/id,reviews/6/rating,reviews/7/author/firstName,reviews/7/author/id,reviews/7/comments,reviews/7/createdAt,reviews/7/id,reviews/7/rating,reviews/8/author/firstName,reviews/8/author/id,reviews/8/comments,reviews/8/createdAt,reviews/8/id,reviews/8/rating,reviews/9/author/firstName,reviews/9/author/id,reviews/9/comments,reviews/9/createdAt,reviews/9/id,reviews/9/rating,roomType,stars,url
247,"Brooklyn, New York, United States",true,true,40.68649,-73.94174,Solar Powered Brownstone w/ treetop terrace,7,"This is the living and dining space which is on the main floor. It's open plan and the kitchen is just to the left. On this floor there are two bedrooms and a bathroom and up the stairs there are three more bedrooms, a second bathroom and a terrace.",https://a0.muscache.com/im/pictures/54c1ccea-b50f-48f4-8d0d-a8a65fb1b17c.jpg?aki_policy=large,https://a0.muscache.com/im/pictures/54c1ccea-b50f-48f4-8d0d-a8a65fb1b17c.jpg?aki_policy=small,859,$859,USD,false,nightly,husband/ wife \r\narchitect/interior designer & artist\r\nwelcoming hosts/ eager travelers,301 Reviews,Identity verified,NaN,James And Siobhan,false,true,521964,true,NaN,2,Joined in April 2011,100%,within an hour,James And Siobhan,2,Phil,245284228,"James has a beautiful apartment in which my family of five had plenty of space. It is very stylish, clean and in a very interesting neighbourhood. James and siobhan were very welcoming and we always felt at home",2023-04-12T19:11:02Z,868135885016915648,5,Masha,18169883,"I had the best experience staying here. The space is just as pictured (gorgeous!), and actually better in person. James & Siobhan are my favorite hosts - they left so many lovely snacks out for our group and we’re always so quick to respond whenever we needed anything. This is by far my favorite Airbnb and it’s in such a lovely area. Highly recommend.",2023-04-09T18:57:18Z,865954648136194431,5,Alisa,2044811,"It was our first time staying in Brooklyn, and James and Siobhan's brownstone apartment made a very suitable base. The townhouse is in a quiet residential area but is still within easy walking distance of a plethora of small restaurants, coffee shops, bodegas and public transportation. The place was clean, well laid out with enjoyable design elements and thoughtfully stocked with treats and basic supplies. The only caveat I would add is that it is not suitable for mobility challenged people. There are in total three decently steep and longish staircases to navigate both to get into the building and once you are in the apartment. <br/>Replies to my inquiries we

In [14]:
file_path = 'data/processed/'
file_name = 'df_nyc_s03.csv'

df_s03.to_csv(file_path+file_name, index=False)

In [15]:
df_s03 = pd.read_csv('data/processed/df_nyc_s03.csv')
df_s03.head(3)

,address,isAvailable,isHostedBySuperhost,location/lat,location/lng,name,numberOfGuests,photos/0/caption,photos/0/pictureUrl,photos/0/thumbnailUrl,pricing/rate/amount,pricing/rate/amountFormatted,pricing/rate/currency,pricing/rate/isMicrosAccuracy,pricing/rateType,primaryHost/about,primaryHost/badges/0,primaryHost/badges/1,primaryHost/badges/2,primaryHost/firstName,primaryHost/hasInclusionBadge,primaryHost/hasProfilePic,primaryHost/id,primaryHost/isSuperHost,primaryHost/languages/0,primaryHost/listingsCount,primaryHost/memberSince,primaryHost/responseRate,primaryHost/responseTime,primaryHost/smartName,primaryHost/totalListingsCount,reviews/0/author/firstName,reviews/0/author/id,reviews/0/comments,reviews/0/createdAt,reviews/0/id,reviews/0/rating,reviews/1/author/firstName,reviews/1/author/id,reviews/1/comments,reviews/1/createdAt,reviews/1/id,reviews/1/rating,reviews/2/author/firstName,reviews/2/author/id,reviews/2/comments,reviews/2/createdAt,reviews/2/id,reviews/2/rating,reviews/3/author/firstName,reviews/3/author/id,reviews/3/comments,reviews/3/createdAt,reviews/3/id,reviews/3/rating,reviews/4/author/firstName,reviews/4/author/id,reviews/4/comments,reviews/4/createdAt,reviews/4/id,reviews/4/rating,reviews/5/author/firstName,reviews/5/author/id,reviews/5/comments,reviews/5/createdAt,reviews/5/id,reviews/5/rating,reviews/6/author/firstName,reviews/6/author/id,reviews/6/comments,reviews/6/createdAt,reviews/6/id,reviews/6/rating,reviews/7/author/firstName,reviews/7/author/id,reviews/7/comments,reviews/7/createdAt,reviews/7/id,reviews/7/rating,reviews/8/author/firstName,reviews/8/author/id,reviews/8/comments,reviews/8/createdAt,reviews/8/id,reviews/8/rating,reviews/9/author/firstName,reviews/9/author/id,reviews/9/comments,reviews/9/createdAt,reviews/9/id,reviews/9/rating,roomType,stars,url
0,"Jersey City, New Jersey, United States",true,false,40.723,-74.03946,Entire Cozy Unit(15 mins to Manhattan),2,NaN,https://a0.muscache.com/im/pictures/miso/Hosting-53775685/original/0812b9a1-2d2e-4725-b51f-881045a939b2.jpeg?aki_policy=large,https://a0.muscache.com/im/pictures/miso/Hosting-53775685/original/0812b9a1-2d2e-4725-b51f-881045a939b2.jpeg?aki_policy=small,99999,"$99,999",USD,false,nightly,Hello all and welcome!\nI love seeing the world and to be able to help people feel right at home.,82 Reviews,Identity verified,NaN,Joes,false,true,245395267,false,中文 (简体),8,Joined in February 2019,NaN,NaN,Joes,14,Miranda,74889676.000,"great location and nice building, loved that parking was included",2022-06-20T19:58:54Z,653626364155945856.000,4,Abhimanyu,4800160.000,"Everything during our stay was perfect and as mentioned in the listing. The location is a 7-min walk to Newport PATH, and right opposite a huge grocery store. Joes was super kind with an early check-in and a late check-out.",2022-05-23T18:57:26Z,633301707521251072.000,5,Robert,162871850.000,Excellent place to stay. Very central fo travel into NYC or if working / travelling in New Jersey. Stayed for 3 months in the apartment. The place was as advertised. Joes is an excellent host also.,2022-05-06T17:28:10Z,620935587400524800.000,5,Yu,303996502.000,It is the best airbnb that I have ever stayed! It gets the great view outside the window. Comfortable and convenient living experience. Everything just settled up for you. It is so great!,2022-01-09T21:55:09Z,536271207399311296.000,5,Mitesh,253080011.000,"A very nice apartment is great locality. The apartment is as listed. i would definitely rate it a 5 star airbnb if not for the pet hair on the carpet and couch. As I had a 9 month old, we were a bit concerned about him having allergies and looking for a clean place. As for Joes, he is very approachable and very easy to communicate.",2021-12-26T20:27:02Z,526079993255321088.000,4.000,Arusha,103758126.000,"Being the first customer, I took a chance in trusting this listing but this apartment exceeded my expectations. It was super clean and also very aesthetically pleasing with a minimalistic vibe.

In [16]:
df_s03.columns

Index(['address', 'isAvailable', 'isHostedBySuperhost', 'location/lat',
       'location/lng', 'name', 'numberOfGuests', 'photos/0/caption',
       'photos/0/pictureUrl', 'photos/0/thumbnailUrl', 'pricing/rate/amount',
       'pricing/rate/amountFormatted', 'pricing/rate/currency',
       'pricing/rate/isMicrosAccuracy', 'pricing/rateType',
       'primaryHost/about', 'primaryHost/badges/0', 'primaryHost/badges/1',
       'primaryHost/badges/2', 'primaryHost/firstName',
       'primaryHost/hasInclusionBadge', 'primaryHost/hasProfilePic',
       'primaryHost/id', 'primaryHost/isSuperHost', 'primaryHost/languages/0',
       'primaryHost/listingsCount', 'primaryHost/memberSince',
       'primaryHost/responseRate', 'primaryHost/responseTime',
       'primaryHost/smartName', 'primaryHost/totalListingsCount',
       'reviews/0/author/firstName', 'reviews/0/author/id',
       'reviews/0/comments', 'reviews/0/createdAt', 'reviews/0/id',
       'reviews/0/rating', 'reviews/1/author/firstName', 

### Step 04 - Getting text columns from dataframe

In [17]:
# Definindo as  colunas de texto
mask_text_cols = "df.columns.str.contains(r'name|about|comments')"
df_txt_cols = df_s03.pipe(remove_cols, [mask_text_cols], inverse=False)
df_txt_cols.sample(1)

,name,primaryHost/about,reviews/0/comments,reviews/1/comments,reviews/2/comments,reviews/3/comments,reviews/4/comments,reviews/5/comments,reviews/6/comments,reviews/7/comments,reviews/8/comments,reviews/9/comments
1,Lux Studio on Wall Street. Heart of Fidi!,Professional working and living in NYC,"This place is exactly as described, perfect for my 1 month stay! Very clean and organized. The location was also quite convenient, close to the supermarket, pharmacy, restaurants, etc. Chris was a wonderful host, extremely attentive and easy to communicate with. Would definitely book again!",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_notxt_cols = df_s03.loc[:, df_s03.columns.difference(df_txt_cols.columns)].copy()
df_notxt_cols.sample(1)

,address,isAvailable,isHostedBySuperhost,location/lat,location/lng,numberOfGuests,photos/0/caption,photos/0/pictureUrl,photos/0/thumbnailUrl,pricing/rate/amount,pricing/rate/amountFormatted,pricing/rate/currency,pricing/rate/isMicrosAccuracy,pricing/rateType,primaryHost/badges/0,primaryHost/badges/1,primaryHost/badges/2,primaryHost/firstName,primaryHost/hasInclusionBadge,primaryHost/hasProfilePic,primaryHost/id,primaryHost/isSuperHost,primaryHost/languages/0,primaryHost/listingsCount,primaryHost/memberSince,primaryHost/responseRate,primaryHost/responseTime,primaryHost/smartName,primaryHost/totalListingsCount,reviews/0/author/firstName,reviews/0/author/id,reviews/0/createdAt,reviews/0/id,reviews/0/rating,reviews/1/author/firstName,reviews/1/author/id,reviews/1/createdAt,reviews/1/id,reviews/1/rating,reviews/2/author/firstName,reviews/2/author/id,reviews/2/createdAt,reviews/2/id,reviews/2/rating,reviews/3/author/firstName,reviews/3/author/id,reviews/3/createdAt,reviews/3/id,reviews/3/rating,reviews/4/author/firstName,reviews/4/author/id,reviews/4/createdAt,reviews/4/id,reviews/4/rating,reviews/5/author/firstName,reviews/5/author/id,reviews/5/createdAt,reviews/5/id,reviews/5/rating,reviews/6/author/firstName,reviews/6/author/id,reviews/6/createdAt,reviews/6/id,reviews/6/rating,reviews/7/author/firstName,reviews/7/author/id,reviews/7/createdAt,reviews/7/id,reviews/7/rating,reviews/8/author/firstName,reviews/8/author/id,reviews/8/createdAt,reviews/8/id,reviews/8/rating,reviews/9/author/firstName,reviews/9/author/id,reviews/9/createdAt,reviews/9/id,reviews/9/rating,roomType,stars,url
327,"New York, United States",true,false,40.738,-73.997,2,Living Room,https://a0.muscache.com/im/pictures/prohost-api/Hosting-30388449/original/d2209902-9d6f-444a-a88f-d9fbd19a99c1.jpeg?aki_policy=large,https://a0.muscache.com/im/pictures/prohost-api/Hosting-30388449/original/d2209902-9d6f-444a-a88f-d9fbd19a99c1.jpeg?aki_policy=small,607,$607,USD,false,nightly,"2,685 Reviews",Identity verified,NaN,Blueground,false,true,107434423,false,English,4805,Joined in December 2016,100%,within an hour,Blueground,5427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entire rental unit,NaN,https://www.airbnb.com/rooms/30388449


In [19]:
arr_texts = df_txt_cols.fillna(' ').values
arr_texts[:5]

array([['Entire Cozy Unit(15 mins to Manhattan)',
        'Hello all and welcome!\nI love seeing the world and to be able to help people feel right at home.',
        'great location and nice building, loved that parking was included',
        'Everything during our stay was perfect and as mentioned in the listing. The location is a 7-min walk to Newport PATH, and right opposite a huge grocery store. Joes was super kind with an early check-in and a late check-out.',
        'Excellent place to stay. Very central fo travel into NYC or if working / travelling in New Jersey. Stayed for 3 months in the apartment. The place was as advertised. Joes is an excellent host also.',
        'It is the best airbnb that I have ever stayed! It gets the great view outside the window. Comfortable and convenient living experience. Everything just settled up for you. It is so great!',
        'A very nice apartment is great locality. The apartment is as listed. i would definitely rate it a 5 star airbnb 

### Step 05 - Defining preprocessing and cleaning operations

In [20]:
if LANG == 'english':
  stanza_lang = 'en'
  spacy_pipe = 'en_core_web_lg'
  
elif LANG == 'portuguese':
  stanza_lang = 'pt'
  spacy_pipe = 'pt_core_news_lg'
  
else:
  print('Antes de continuar, defina a variavel LANG = "portuguese" | "english"')

In [21]:
from urllib.request import Request, urlopen 

req = Request(f'https://countwordsfree.com/stopwords/{LANG}/txt')
req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
content = urlopen(req)
stop_words = pd.read_csv(content, header=None)
stop_words = stop_words[0].values
# These words are important for problem. We don't want to remove them.
excluding = ['against', 'not', 'don', 'don\'t','ain', 'are', 'aren\'t', 'could', 'couldn\'t',
             'did', 'didn\'t', 'does', 'doesn\'t', 'had', 'hadn\'t', 'has', 'hasn\'t', 
             'have', 'haven\'t', 'is', 'isn\'t', 'might', 'mightn\'t', 'must', 'mustn\'t',
             'need', 'needn\'t','should', 'shouldn\'t', 'was', 'wasn\'t', 'were', 
             'weren\'t', 'won\'t', 'would', 'wouldn\'t']
stop_words = [sw for sw in stop_words if sw not in excluding]

In [22]:
!python -m spacy download {spacy_pipe}


2023-06-01 22:22:56.356247: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 22:22:57.358219: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-01 22:22:58.733741: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-01 22:22:58.734200: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [26]:
from functools import cache
from typing import Union
import spacy
spacy.prefer_gpu()
import stanza


stanza.download(stanza_lang)
nlp_spacy = spacy.load(spacy_pipe, disable=['parser', 'ner', 'textcat']) 
spacy_type = type(nlp_spacy)

@cache
def preprocess_txt(textraw:str, nlp_obj:Union[spacy_type, stanza.pipeline.core.Pipeline], lemma=False) -> str:
  '''
  Realiza a limpeza e o pre processamento de um dado texto.
  As etapas de pré-processamento a serem executadas são:
  * Conversão para Minúsculas.
  * Remocao de tags HTML.
  * Remocao de pontuacoes.
  * Remocao de algarismos numericos e numeros de telefone.
  * Remocao de multiplos espacos.
  * Remocao de URLs e E-mails: são removidos, pois eles não adicionam significado ao texto.
  * Substituicao de Emojis: Substitui emojis usando um dicionário predefinido contendo emojis junto com seu significado. (por exemplo: ":)" para "sorriso")
  * Substituindo nomes de usuário: Substitua @Usernames pela palavra "USER". (por exemplo: "@nick_name" para "USER")
  * Remocao de ascentuacao.
  * Removendo Não-Alfabetos: Substitui caracteres (exceto Dígitos e Alfabetos) por um espaço.
  * Removendo letras consecutivas: 3 ou mais letras consecutivas são substituídas por 2 letras. (por exemplo: "Muitoooo" para "Muitoo")
  * Remoção de palavras curtas: palavras com comprimento menor que 2 são removidas.
  * Removendo Stopwords: Stopwords são as palavras que não adicionam muito significado a uma frase. Eles podem ser ignorados com segurança sem sacrificar o significado da frase. (por exemplo: "the", "he", "have")
  * Lematização: A lematização é o processo de converter uma palavra em sua forma básica. (por exemplo: “Great” para “Good”). A lematização geralmente retorna palavras válidas (que existem) enquanto as técnicas de stemming retornam (na maioria das vezes) palavras abreviadas

  Parameters:
    textraw (str): texto bruto o qual sofrera as operacoes de limpeza e pre processamento.
    nlp_obj (spacy.lang.pt.Portuguese | spacy.lang.en.English | stanza.pipeline.core.Pipeline): - objeto que indica a biblioteca a ser utilizada para a etapa de tokenizacao, remocao de stop words e lemmatizacao.
    lemma (boolean): (opcional) se deve substituir as palavras originais do texto por seu lemma ou nao. Default: False

  Returns:
    wordsclean (pd.Dataframe): dataframe sem as colunas indicadas na máscara
  '''
  import re
  from unidecode import unidecode
  # from gensim.utils import simple_preprocess
  
  global stop_words

  # Definindo padrões Regex
  # urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
  # userPattern       = '@[^\s]+'
  alphaPattern      = "[^a-zA-Z0-9]"
  sequencePattern   = r"(.)\1\1+" # letras consecutivas
  seqReplacePattern = r"\1\1"
  
  # Definindo dicionario de emojis e seus respectivos significados
  emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
            ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
            ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
            ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
            '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
            '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
            ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

   
  # ##################
  # Tentativa de aplicação de técnica para correção de palavras com erro de escrita, 
  # mas o resultado não foi considerado satisfatório.
  # ##################
  # from gensim.utils import simple_tokenize
  # from spellchecker import SpellChecker
  # spell = SpellChecker(language='pt')
  # wordlist = list(simple_tokenize(textraw))
  # misspelled = spell.unknown(wordlist)
  # correctedtext = [spell.correction(word) for word in misspelled]
  # print(correctedtext)

  textclean = neat_preprocessing(textraw)  
  for emoji in emojis.keys():
      textclean = textclean.replace(emoji, "EMOJI" + emojis[emoji])  
  textclean = re.sub(sequencePattern, seqReplacePattern, textclean)
  textclean = unidecode(textclean)
  textclean = re.sub(alphaPattern, " ", textclean)
  wordsclean = ''

  if nlp_obj.__module__ == 'spacy.lang.en' or nlp_obj.__module__ == 'spacy.lang.pt':
    doc = nlp_obj(textclean)
    for token in doc:      
      if not token.is_stop and len(token) > 1:
        if lemma and token.pos_ == 'VERB':
          wordsclean += token.lemma_+' '
        else:
          wordsclean += token.text + ' '

  elif isinstance(nlp_obj, stanza.pipeline.core.Pipeline):
    doc = nlp_obj(textclean)
    for sent in doc.sentences:
      for word in sent.words:   
        if word.text not in stop_words and len(word.text) > 1:   
          if lemma and word.upos == 'VERB':
            wordsclean += word.lemma +' '
          else:
            wordsclean += word.text + ' '
  else:
      raise BaseException('Instancia nao reconhecida do objeto "nlp_obj". Opcoes: spacy | stanza')
    
  return wordsclean[:-1]


INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


### Step 06 - Tests

In [24]:
arr_texts[133]

array(['Beautiful house in the heart of Williamsburg',
       'There are 3 places that we call home. Brooklyn, the 1000 Islands &  Puerto Rico. We split our time between all 3 and have created homes that we are happy to share with you.',
       'Julia’s place was exactly what we needed. Great spot to work remotely and explore Brooklyn and the city. The location is close to everything including transit. Beds were comfortable and the home had lots of space to spread out. Both Julia and her cleaning staff were very friendly and kind, quick to respond. <br/>As others mentioned this is a louder location on a busy street. Earplugs are provided and necessary at night which was fine for us. Worth noting: there is no microwave or way to reheat food so if you plan to cook or stay for an extended period of time it’s something to consider.',
       'Great spot in a perfect location! The house was spacious with a well appointed kitchen and plenty of room for our family.',
       "We stayed at Julia

#### Teste Spacy

In [27]:
%%time

postprocess = []
for rawtxt in arr_texts[133]:
  if len(rawtxt):
    postprocess.append(preprocess_txt(rawtxt, nlp_obj=nlp_spacy, lemma=True))

postprocess

CPU times: user 227 ms, sys: 1.4 ms, total: 228 ms
Wall time: 235 ms


['beautiful house heart williamsburg',
 'places home brooklyn islands puerto rico split time create homes happy share',
 'julia place exactly need great spot work remotely explore brooklyn city location close include transit beds comfortable home lots space spread julia clean staff friendly kind quick respond mention louder location busy street earplugs provide necessary night fine worth note microwave way reheat food plan cook stay extended period time consider',
 'great spot perfect location house spacious appoint kitchen plenty room family',
 'stay julias home weeks home renovate home central get williamsburg close best restaurants williamsburg go eat night able multiple daily walks dogs mccarren park day home huge clean work home ample room meetings time yard huge plus play time dogs julia incredibly responsive kind let check earlier plan highly recommend stay',
 'julia responsible kind wellprepared house',
 'place gem humble exterior give way beautiful bespoke interior thoughtful 

####  Teste Stanza

In [ ]:
%%time
########## Stanza defined's processors for this task:
# tokenize - Segments a Document into Sentences, each containing a list of Tokens.
# mwt - Expands multi-word tokens (MWTs) into multiple words when they are predicted by the tokenizer.Fix contractions.
# pos - UPOS, XPOS, and UFeats annotations are accessible through Word’s properties `pos`, `xpos`, and `ufeats`.
# lemma - Perform lemmatization on a Word using the `Word.text` and `Word.upos` values. The result can be accessed as `Word.lemma`. 
nlp_stanza = stanza.Pipeline(lang=stanza_lang, processors='tokenize,mwt,pos,lemma', use_gpu=True, download_method=None, verbose=False, max_cache_size=2)
postprocess = []
for rawtxt in arr_texts[133]:
  if len(rawtxt):
    postprocess.append(preprocess_txt(rawtxt, nlp_obj=nlp_stanza, lemma=True))

postprocess

CPU times: user 2.36 s, sys: 570 ms, total: 2.93 s
Wall time: 6.28 s


['beautiful house heart williamsburg ',
 'places brooklyn islands puerto rico split time create homes happy share ',
 'julia place need great spot work remotely explore brooklyn city location close include transit beds comfortable lots space spread julia cleaning staff friendly kind quick respond mention louder location busy street earplugs night fine worth note microwave reheat food plan cook stay extended period time ',
 'great spot perfect location house spacious appoint kitchen plenty room family ',
 'stay julias weeks renovate central williamsburg close restaurants williamsburg eat night multiple daily walks dogs mccarren park day huge clean work ample room meetings time yard huge play time dogs julia incredibly responsive kind check earlier plan highly recommend stay ',
 'julia responsible kind wellprepare house ',
 'place gem humble exterior beautiful bespoke interior thoughtful creative touches living room kitchen backyard area open spacious surprising nyc bedrooms smaller host

### Step 07 - Applying preprocessing operations on dataframe text columns

#### Using SpaCy

In [28]:
%%time
df_txt_cols_processed_spacy = parallel_applymap(df_txt_cols, preprocess_txt, worker_count=10, na_action='ignore', nlp_obj=nlp_spacy, lemma=True)

  0%|          | 0/672 [00:00<?, ?it/s]

  0%|          | 0/684 [00:00<?, ?it/s]

  0%|          | 0/672 [00:00<?, ?it/s]

  0%|          | 0/672 [00:00<?, ?it/s]

  0%|          | 0/672 [00:00<?, ?it/s]

...

  0%|          | 0/672 [00:00<?, ?it/s]

  0%|          | 0/672 [00:00<?, ?it/s]

  0%|          | 0/672 [00:00<?, ?it/s]

  0%|          | 0/672 [00:00<?, ?it/s]

CPU times: user 1min 14s, sys: 5.46 s, total: 1min 20s
Wall time: 1min 25s


In [29]:
df_txt_cols_processed_spacy.loc[133:134]

,name,primaryHost/about,reviews/0/comments,reviews/1/comments,reviews/2/comments,reviews/3/comments,reviews/4/comments,reviews/5/comments,reviews/6/comments,reviews/7/comments,reviews/8/comments,reviews/9/comments
133,beautiful house heart williamsburg,places home brooklyn islands puerto rico split time create homes happy share,julia place exactly need great spot work remotely explore brooklyn city location close include transit beds comfortable home lots space spread julia clean staff friendly kind quick respond mention louder location busy street earplugs provide necessary night fine worth note microwave way reheat food plan cook stay extended period time consider,great spot perfect location house spacious appoint kitchen plenty room family,stay julias home weeks home renovate home central get williamsburg close best restaurants williamsburg go eat night able multiple daily walks dogs mccarren park day home huge clean work home ample room meetings time yard huge plus play time dogs julia incredibly responsive kind let check earlier plan highly recommend stay,julia responsible kind wellprepared house,place gem humble exterior give way beautiful bespoke interior thoughtful creative touches need home living room kitchen backyard area open spacious surprising nyc bedrooms smaller hosts create storage space etc speak hosts julia incredibly responsive stay know start booking process work reliable helpful people great trip hope stay,julias place truly little oasis outdoor area nice rarely find new york house super spacious modern locate great location right mccarren park near williamsburg shops restaurants stay time visit nyc,wonderful place comfy wellappointed spacious open thoughtful amenities minutes walk mccarren park minutes foods pharmacy ideal location place make feel like safe home open floor plan downstairs make great living space projector tv setup ideal son game heaven outdoor space calming delightful cooks kitchen wellappointe lay outtheres calm sanctuary inside bustling brooklyn right door everchange hipster parade love stay,space lovely room relax breath walk distance love stay hope future,unassuming street facade lovely quiet oasis heart williamsburg husband grown daughters gather julia place home base great space offer comforts home extremely comfortable amenities story spot bedrooms comfy beds need williamsburg adventure walk door walk mccarren park walk ferry stop travel ferry th street river manhattan walk shops bedford ave park car street safe secure lot plenty street parking julia extremely responsive easy work hope enjoy significant outdoor deck find,great place stay
134,beautiful bdrm bath private luxury townhouse,lawyer enjoy garden read,great apartment greenwich village area manhattan location convenient walk get subway call uber feel completely safe come apartment evenings entrance light near major street apartment spacious nyc comfortable beds baths stock plenty towels good showers hairdryers nice kitchen include size fridge coffee maker plenty glasses plates vike range need cook manhattan apartment huge windows nice views roomswe family plenty space bit street noise early morning days sanitation trucks pick evenings mornings apartment completely quiet dan great host responsive send long helpful list restaurants arrive,great location stay recommend place little minor issues fix beds comfortable,great location trip nyc,fabulous location great nyc lots great bars restaurants accessible manhattan key assets lovely apartment maintenance need eg leak roof inaccessible terrace consequently despite show list photographs dollar price pay hope better maintain property dan communicate provide great tips local venues visit etc raise legitimate concerns terrace leak roof go pretty quiet recommend property definitively worth seek assurance feedback provide recent guests address,dan give lot info local guidance responsive keypad entry great lose keys come early area great like,dan responsive considerate host christmas 

#### Using Stanza

In [ ]:
%%time
nlp_stanza = stanza.Pipeline(lang=stanza_lang, processors='tokenize,mwt,pos,lemma', use_gpu=True, download_method=None, verbose=False, max_cache_size=2)
df_txt_cols_processed_stanza = parallel_applymap(df_txt_cols, preprocess_txt, worker_count=10, na_action='ignore', nlp_obj=nlp_stanza, lemma=True)

  0%|          | 0/684 [00:00<?, ?it/s]

  0%|          | 0/672 [00:00<?, ?it/s]

  0%|          | 0/672 [00:00<?, ?it/s]

  0%|          | 0/672 [00:00<?, ?it/s]

  0%|          | 0/672 [00:00<?, ?it/s]

...

  0%|          | 0/672 [00:00<?, ?it/s]

  0%|          | 0/672 [00:00<?, ?it/s]

  0%|          | 0/672 [00:00<?, ?it/s]

  0%|          | 0/672 [00:00<?, ?it/s]

CPU times: user 7min 45s, sys: 30.6 s, total: 8min 15s
Wall time: 6min 48s


#### Comparando o texto não processado com o processado

In [30]:
df_txt_cols.loc[0:5]

,name,primaryHost/about,reviews/0/comments,reviews/1/comments,reviews/2/comments,reviews/3/comments,reviews/4/comments,reviews/5/comments,reviews/6/comments,reviews/7/comments,reviews/8/comments,reviews/9/comments
0,Entire Cozy Unit(15 mins to Manhattan),Hello all and welcome!\nI love seeing the world and to be able to help people feel right at home.,"great location and nice building, loved that parking was included","Everything during our stay was perfect and as mentioned in the listing. The location is a 7-min walk to Newport PATH, and right opposite a huge grocery store. Joes was super kind with an early check-in and a late check-out.",Excellent place to stay. Very central fo travel into NYC or if working / travelling in New Jersey. Stayed for 3 months in the apartment. The place was as advertised. Joes is an excellent host also.,It is the best airbnb that I have ever stayed! It gets the great view outside the window. Comfortable and convenient living experience. Everything just settled up for you. It is so great!,"A very nice apartment is great locality. The apartment is as listed. i would definitely rate it a 5 star airbnb if not for the pet hair on the carpet and couch. As I had a 9 month old, we were a bit concerned about him having allergies and looking for a clean place. As for Joes, he is very approachable and very easy to communicate.","Being the first customer, I took a chance in trusting this listing but this apartment exceeded my expectations. It was super clean and also very aesthetically pleasing with a minimalistic vibe. This is definitely a luxury apartment building and in a convenient location - right by Shoprite and near the waterfront and path! The host was also very responsive about any questions. Thanks for a great stay!",NaN,NaN,NaN,NaN
1,Lux Studio on Wall Street. Heart of Fidi!,Professional working and living in NYC,"This place is exactly as described, perfect for my 1 month stay! Very clean and organized. The location was also quite convenient, close to the supermarket, pharmacy, restaurants, etc. Chris was a wonderful host, extremely attentive and easy to communicate with. Would definitely book again!",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3 Studio Double Queen at Refinery Hotel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Home-1 near NYC (Filming/Weddings/Retreats),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Artist's Loft,"Producer/Director with award-winning independent production company, http://www.IndustrialStrengthNYC.com",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Cozy, quiet, private 1 br apt NO SMOKING ALLOWED",NaN,Thank you so much Johanna. I couldn’t be more grateful for how well you accommodated my parents. It means the world to me that they are happy and they told me that you were a wonderful host. 🙏,"DO NOT BOOK !!!!!!!!<br/>By far the worst airbnb experience I have ever had. The woman repeatedly harassed me whenever she knew I was alone and I think it was because she knew I wasn’t from the area. After the first night I stayed she lied to Airbnb about what I was doing with the apartment and tried to have me kicked out without a refund. I spoke to Airbnb and explained to them what was happening and she immediately switched up her story and asked me to stay. I explained to Airbnb and her that I was no longer comfortable staying because I felt she was trying to take advantage of me and she didn’t even have the decency to return my calls or messages even though I messaged her through the Airbnb app. Whenever I had company over she was quiet but whenever she knew I was alone they would lie and try to finesse me for money. She insisted on trying to add random nonsense cleaning fees over things that i had absolutely nothing to do with, at one point I offered her to come inside and check around and she refused to, but she still went to Airbnb accusing me of things that clearly were not happening. Even though Airbnb told her she was not to have any more contact with me she came repeatedl

In [ ]:
df_txt_cols_processed_spacy.loc[0:5]

,name,primaryHost/about,reviews/0/comments,reviews/1/comments,reviews/2/comments,reviews/3/comments,reviews/4/comments,reviews/5/comments,reviews/6/comments,reviews/7/comments,reviews/8/comments,reviews/9/comments
0,entire cozy unit mins manhattan,love people feel,great location nice building love parking include,stay perfect mention listing location min walk newport path huge grocery store joes super kind early checkin late checkout,excellent place stay central fo travel nyc work travel jersey stay months apartment place advertise joes excellent host,airbnb stay great view window comfortable convenient living experience settle great,nice apartment great locality apartment list rate star airbnb pet hair carpet couch month bit concerned allergies clean place joes approachable easy communicate,customer chance trust listing apartment exceed expectations super clean aesthetically pleasing minimalistic vibe luxury apartment building convenient location shoprite waterfront path host responsive questions great stay,NaN,NaN,NaN,NaN
1,lux studio wall street heart fidi,professional work live nyc,place perfect month stay clean organize location convenient close supermarket pharmacy restaurants chris wonderful host extremely attentive easy communicate book,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,studio double queen refinery hotel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,nyc filming weddings retreat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,artists loaf,producer director awardwin independent production company http wwindustrialstrengthnyccom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,cozy quiet private br apt smoking allow,NaN,johanna couldn grateful accommodate parents happy tell wonderful host EMOJI,book worst airbnb experience woman repeatedly harass know know wasn area night stay lie airbnb apartment kick refund speak airbnb explain happen switch story ask stay explain airbnb longer comfortable stay feel advantage didn decency return calls messages message airbnb app company quiet know lie finesse money insist add random nonsense cleaning fees absolutely point offer check refuse airbnb accuse happen airbnb tell contact repeatedly state leave refund call airbnb tell couldn time bring husband clear attempt intimidate small foreign woman point police involve tell wrong airbnb finally refund rest stay allow leave liar finesse people money police airbnb run illegal apartment tiny woman rude disrespectful liar separate apartment host upstairs staircase door open anytime mention listing woman nicest person place isn worth price chance meet woman deal lie price,place cozy clean enjoy tv living room bedroom bathroom nice stock lots shampoo conditioner body wash perfect incase forget bring toiletries stay complimentary coffee hot chocolate lovely touch host accommodating check easy recommend place short stay stars,johanna place clean comfortable communication johanna expectations mind ease travel newborn son accommodate request,stay wonderful enjoy neighborhood location close nightlife groceries need johanna great host,perfect location enjoy weekend bronx close mass transit bring city great places eat nearby walking distance apartment clean cozy,fine guest,la estancia es muy comoda limpia el lugar es muy tranquilo el lugar muy limpio practico,NaN,NaN


#### Salvando o dataset preprocessado

In [32]:
df_final = pd.concat([df_notxt_cols, df_txt_cols_processed_spacy], axis=1)
df_final.loc[0:5]

,address,isAvailable,isHostedBySuperhost,location/lat,location/lng,numberOfGuests,photos/0/caption,photos/0/pictureUrl,photos/0/thumbnailUrl,pricing/rate/amount,pricing/rate/amountFormatted,pricing/rate/currency,pricing/rate/isMicrosAccuracy,pricing/rateType,primaryHost/badges/0,primaryHost/badges/1,primaryHost/badges/2,primaryHost/firstName,primaryHost/hasInclusionBadge,primaryHost/hasProfilePic,primaryHost/id,primaryHost/isSuperHost,primaryHost/languages/0,primaryHost/listingsCount,primaryHost/memberSince,primaryHost/responseRate,primaryHost/responseTime,primaryHost/smartName,primaryHost/totalListingsCount,reviews/0/author/firstName,reviews/0/author/id,reviews/0/createdAt,reviews/0/id,reviews/0/rating,reviews/1/author/firstName,reviews/1/author/id,reviews/1/createdAt,reviews/1/id,reviews/1/rating,reviews/2/author/firstName,reviews/2/author/id,reviews/2/createdAt,reviews/2/id,reviews/2/rating,reviews/3/author/firstName,reviews/3/author/id,reviews/3/createdAt,reviews/3/id,reviews/3/rating,reviews/4/author/firstName,reviews/4/author/id,reviews/4/createdAt,reviews/4/id,reviews/4/rating,reviews/5/author/firstName,reviews/5/author/id,reviews/5/createdAt,reviews/5/id,reviews/5/rating,reviews/6/author/firstName,reviews/6/author/id,reviews/6/createdAt,reviews/6/id,reviews/6/rating,reviews/7/author/firstName,reviews/7/author/id,reviews/7/createdAt,reviews/7/id,reviews/7/rating,reviews/8/author/firstName,reviews/8/author/id,reviews/8/createdAt,reviews/8/id,reviews/8/rating,reviews/9/author/firstName,reviews/9/author/id,reviews/9/createdAt,reviews/9/id,reviews/9/rating,roomType,stars,url,name,primaryHost/about,reviews/0/comments,reviews/1/comments,reviews/2/comments,reviews/3/comments,reviews/4/comments,reviews/5/comments,reviews/6/comments,reviews/7/comments,reviews/8/comments,reviews/9/comments
0,"Jersey City, New Jersey, United States",true,false,40.723,-74.03946,2,NaN,https://a0.muscache.com/im/pictures/miso/Hosting-53775685/original/0812b9a1-2d2e-4725-b51f-881045a939b2.jpeg?aki_policy=large,https://a0.muscache.com/im/pictures/miso/Hosting-53775685/original/0812b9a1-2d2e-4725-b51f-881045a939b2.jpeg?aki_policy=small,99999,"$99,999",USD,false,nightly,82 Reviews,Identity verified,NaN,Joes,false,true,245395267,false,中文 (简体),8,Joined in February 2019,NaN,NaN,Joes,14,Miranda,74889676.000,2022-06-20T19:58:54Z,653626364155945856.000,4,Abhimanyu,4800160.000,2022-05-23T18:57:26Z,633301707521251072.000,5,Robert,162871850.000,2022-05-06T17:28:10Z,620935587400524800.000,5,Yu,303996502.000,2022-01-09T21:55:09Z,536271207399311296.000,5,Mitesh,253080011.000,2021-12-26T20:27:02Z,526079993255321088.000,4.000,Arusha,103758126.000,2021-12-22T21:29:10Z,523212165154174464.000,5.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entire rental unit,4.670,https://www.airbnb.com/rooms/53775685,entire cozy unit mins manhattan,hello welcome love see world able help people feel right home,great location nice building love parking include,stay perfect mention listing location min walk newport path right opposite huge grocery store joes super kind early checkin late checkout,excellent place stay central fo travel nyc work travel new jersey stay months apartment place advertise joes excellent host,best airbnb stay get great view outside window comfortable convenient living experience settle great,nice apartment great locality apartment list definitely rate star airbnb pet hair carpet couch month old bit concerned have allergies look clean place joes approachable easy communicate,customer take chance trust listing apartment exceed expectations super clean aesthetically pleasing minimalistic vibe definitely luxury apartment building convenient location right shoprite near waterfront path host responsive questions thanks great stay,NaN,NaN,NaN,NaN
1,"New York, United States",true,false,40.706,-74.0092,2,NaN,https://a0.muscache.com/im/pictures/e2388507-1f5f-4000-aa1b-d3b2279e682a.jpg?aki_policy=large,https://a0.muscache.com/im/pictures

In [33]:
file_path = 'data/processed/'
# file_name = 'dataset_airbnb-processed_2023-04-04_01-45-49-997.csv'
file_name = 'dataset_airbnb-processed-spacy_2023-04-13_03-28-09-439.csv'

df_final.to_csv(file_path+file_name, index=False)